In [1]:
 from julia import Julia
jl = Julia(sysimage = "/home/callum/ASF/Fitting/sys_model.so") #loading sys image

In [2]:
import matplotlib.pyplot as plt

import os
import tempfile
import numpy as np
import scipy as sp


from pyabc import ABCSMC, RV, Distribution, LocalTransition, MedianEpsilon, SingleCoreSampler
from pyabc.visualization import plot_data_callback, plot_kde_2d
from pyabc.populationstrategy import AdaptivePopulationSize
import pyabc
from scipy import stats
import random as rd
from brokenaxes import brokenaxes

In [3]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

In [4]:
plt.style.use('seaborn-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# Fitting ODE model and deciding best!

In [12]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/ODE_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/ODE_FIT.jl') #loading files with our model!

/home/callum/.local/lib/python3.10/site-packages/julia/core.py:709: FutureWarning: Accessing `Julia().<name>` to obtain Julia objects is deprecated.  Use `from julia import Main; Main.<name>` or `jl = Julia(); jl.eval('<name>')`.
  warnings.warn(


<PyCall.jlwrap Main.SIR_ODE>

In [14]:
#Loading the four models!
model1 = jl.SIR_ODE.model_1 #Frequency
model2 = jl.SIR_ODE.model_2 #Pure Density
model3 = jl.SIR_ODE.model_3 #Sigmoid Density
model4 = jl.SIR_ODE.model_4 #Sqrt Density

models  =[model1,model2,model3,model4]

In [ ]:
#The observations!
obs = jl.SIR_ODE.observation

#Distance function used for fitting
distance = jl.SIR_ODE.distance


In [ ]:
#Fitting two params P1 - transmission co-efficent, P2 - corpse infectivity modifier
parameter_prior = Distribution(p1=RV("uniform", 0.0, 1.0),p2=RV("uniform", 0.0, 1.0))

#sample params to test functions all work and pre-compile
sample_par = {"p1": 0.1,"p2": 0.5} 

n = 4 #number of models we are looking at!
param_list = [parameter_prior]*n

In [ ]:
distance(model1(sample_par),obs)
distance(model2(sample_par),obs)
distance(model3(sample_par),obs)
distance(model4(sample_par),obs);

In [ ]:
abc= ABCSMC(
    models=model4,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size =  AdaptivePopulationSize(1000, 0.15,max_population_size = 1000),
    sampler = SingleCoreSampler(),
)



# Tau Leaping Homogeneous 

In [12]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HOMO_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HOMO_FIT.jl') #loading files with our model!
model_int = jl.SIR_TAU_S.model_int

/home/callum/.local/lib/python3.10/site-packages/julia/core.py:709: FutureWarning: Accessing `Julia().<name>` to obtain Julia objects is deprecated.  Use `from julia import Main; Main.<name>` or `jl = Julia(); jl.eval('<name>')`.
  warnings.warn(


<PyCall.jlwrap Main.SIR_TAU_S>

In [13]:
model_int = jl.SIR_TAU_S.model_int

In [ ]:
model1 = jl.SIR_TAU_S.model_1 #Frequency
model2 = jl.SIR_TAU_S.model_2 #Pure Density
model3 = jl.SIR_TAU_S.model_3 #Sigmoid Density
model4 = jl.SIR_TAU_S.model_4 #Sqrt Density

#models_h  =[model1_h,model2_h,model3_h,model4_h]

In [ ]:
distance(model1_h (sample_par),obs)
distance(model2_h (sample_par),obs)
distance(model3_h (sample_par),obs)
distance(model4_h (sample_par),obs);

In [ ]:
abc = ABCSMC(
    models=model4_h,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size =  AdaptivePopulationSize(1000, 0.15,max_population_size = 1000),
    sampler = SingleCoreSampler(),
)

In [ ]:

abc_id = abc.new("sqlite:////tmp/mjp.db", obs)


In [ ]:

h4_h = abc.run(minimum_epsilon=0.5, max_nr_populations=15)


# Tau Hetro

In [ ]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HETRO_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HETRO_FIT.jl') #loading files with our model!

In [ ]:
model1= jl.SIR_TAU_M.model_1 #Frequency
#model2 = jl.SIR_TAU_M.model_2 #Pure Density
#model3 = jl.SIR_TAU_M.model_3 #Sigmoid Density
model4 = jl.SIR_TAU_M.model_4 #Sqrt Density

In [ ]:
#Fitting two params P1 - transmission co-efficent, P2 - corpse infectivity modifier
parameter_prior = Distribution(p1=RV("uniform", 0.0, 1.0),p2=RV("uniform", 0.0, 0.25),p3=RV("uniform", 0.0, 1.0),)

#sample params to test functions all work and pre-compile
sample_par = {"p1": 0.1,"p2": 0.05, "p3": 0.5} 

n = 4 #number of models we are looking at!
param_list = [parameter_prior]*n

In [ ]:
distance(model1_c (sample_par),obs)
distance(model2_c (sample_par),obs)
distance(model3_c (sample_par),obs)
distance(model4_c (sample_par),obs);

In [ ]:
abc = ABCSMC(
    models=model1_c,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size =  AdaptivePopulationSize(100, 0.15,max_population_size = 100),
    sampler = SingleCoreSampler(),
)




In [ ]:
abc_id = abc.new("sqlite:////tmp/mjjp.db", obs)


In [ ]:
h1n = abc.run(minimum_epsilon=0.5, max_nr_populations=6)
